In [ ]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [8]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('int_id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v))

class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    borehole = Column(String(32), ForeignKey('Boreholes.id'),
                           primary_key=True)
    # Note that this could be a numeric ID as well
    id = Column(Integer, primary_key=True)
    description = Column(String(32))

In [9]:
engine = create_engine('sqlite:///test_db.db', echo=True)

In [10]:
Base.metadata.create_all(engine)

2020-11-29 20:03:47,746 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-29 20:03:47,748 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 20:03:47,752 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-29 20:03:47,754 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 20:03:47,757 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("machine")
2020-11-29 20:03:47,759 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 20:03:47,762 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("sensor")
2020-11-29 20:03:47,764 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 20:03:47,767 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-11-29 20:03:47,769 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 20:03:47,771 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-11-29 20:03:47,772 INFO sqlalchemy.engine.base.Engine ()
2020-11-29 20:0

In [11]:
m = Borehole(id='F10')

In [12]:
m.intervals_values = {0: 'stuff', 1: 'junk'}

In [13]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [14]:
with session_scope() as session:
    session.add(m)

2020-11-29 20:04:45,688 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-29 20:04:45,692 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-11-29 20:04:45,693 INFO sqlalchemy.engine.base.Engine ('F10',)
2020-11-29 20:04:45,697 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (borehole, id, description) VALUES (?, ?, ?)
2020-11-29 20:04:45,698 INFO sqlalchemy.engine.base.Engine (('F10', 0, 'stuff'), ('F10', 1, 'junk'))
2020-11-29 20:04:45,700 INFO sqlalchemy.engine.base.Engine COMMIT
